# 1st Stage Panel Data Cleaning

This notebook performs the first round of data cleaning tasks on the final long form data produced from the Aggregate Listings Data notebook. These tasks include creating leads and lags, general formatting, filling N/A values, and identifying missing neighborhoods. It then saves the cleaned data into a compressed csv.gz file.

In [1]:
import numpy as np
import pandas as pd
import os
import operator # This allows one to pass operators into a Python function
import mpu # For distance calculation
from scipy import stats # Used to find modal value of geographic values
import time

_______

In [2]:
# Select city to work with

city_folder = '/united-states_portland'
city_abbrev = 'POR'

# Preliminary Cleaning

In [3]:
km_per_mi = 1.60934 #for distance conversion

In [4]:
# Universal directory setup
cwd1 = os.getcwd() 

# Go up one directory level
os.chdir('..')
cwd2 = os.getcwd()

csv_raw_path = cwd2 + '/1. Download and compile data/'
csv_save_path = cwd2 + '/2. Clean data/'

# Revert to preliminary directory
os.chdir(cwd1)

In [5]:
# Read concatenated data
os.chdir(csv_raw_path)

listings_df = pd.read_csv(city_abbrev + '_Data_longALL.csv.gz', low_memory=False)

# Switch to other folder for saving data
os.chdir(csv_save_path)

# Show a snapshot of the dataframe
print(listings_df.iloc[:6,:6])

        id  month  List_month last_scraped     host_id       host_name
0  4986792      0           1   2015-09-02   9165660.0         Cynthia
1  3883718      0           1   2015-09-02  13686663.0         Mallory
2  7092722      0           1   2015-09-02  13322905.0           Sarah
3  4475369      0           1   2015-09-02   4006487.0        Kathleen
4  5904142      0           1   2015-09-02  19052765.0  Melanie & Dirk
5  2455288      0           1   2015-09-02  11021785.0             Amy


------

### Destringing prices

In [6]:
def destring_price(var):
    """
    Destrings a passed variable.
    """
    listings_df.loc[:, var] = listings_df[var].replace('[\$,]', '', regex=True).astype(float)

# This loop destrings price variables
for var in ['price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']:
    destring_price(var)

### Date formatting


In [7]:
def format_dates(var):
    """
    This function converts date variables into datetime format.
    """
    listings_df.loc[:, var] = pd.to_datetime(listings_df[var])
    
# ===============================================================    

def dates_diff(var_name, var1, var2):
    """
    Computes the difference between two date variables and assigns
    the difference to a new variable of given name.
    """
    listings_df.loc[:, var_name] = listings_df[var1] - listings_df[var2]

In [8]:
# Create a loop of date formatting
for date_vars in ['last_scraped', 'host_since', 'first_review', 'last_review']:
    format_dates(date_vars)
    
# Set the 'scrape_batch' as a modal date for a file being scraped in a CSV

for m in listings_df['month'].unique():
    listings_df.loc[listings_df['month'] == m, 'scrape_batch'] = listings_df[listings_df['month'] == m]['last_scraped'].mode().values[0]
    listings_df.loc[:, 'scrape_batch'] = pd.to_datetime(listings_df['scrape_batch'])
    
# Create a Year-Month value for the scrape batch, this is largely used for graphing where one needs to aggregate by year-month
listings_df.loc[:,"batch_YRMO"] = pd.to_datetime(listings_df['scrape_batch']).dt.to_period('M')    

# Calculate different date differences
dates_diff('days_since_rev', 'last_scraped', 'last_review')
dates_diff('days_since_first_rev', 'last_scraped', 'first_review')
dates_diff('host_length', 'last_scraped', 'host_since')

### Leads and lag creation

In [9]:
def create_lagsleads(var, lag_range, df, title):
    
    """
    This function creates lag variables within a given range 
    for a given variable within a given dataframe. The title of these lag 
    variables is specified by title.
    """
    
    df = df.sort_values(by = ['id', 'month'])
    
    for i in range(-lag_range, lag_range + 1):
        
        if i == 0:
            continue
        
        if i < 0:
            df.loc[:, title + "lead" + str(abs(i)) ] = df.groupby('id')[var].shift(i)
                
        if i > 0: 
            df.loc[:, title + "lag" + str(abs(i)) ] = df.groupby('id')[var].shift(i)
            
    return df

listings_df = create_lagsleads('List_month', 12, listings_df, "List")
listings_df = create_lagsleads('availability_60', 12, listings_df, "avail60")
listings_df = create_lagsleads('availability_90', 12, listings_df, "avail90")
listings_df = create_lagsleads('availability_365', 12, listings_df, "avail365")

/Users/Lauri/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


In [10]:
# Calculating a cleaned version of changes in the number of reviews a property has.

listings_df = listings_df.reset_index(drop=True)

listings_df.loc[:, 'max_lag'] = listings_df.groupby(['id'])['number_of_reviews'].rolling(4).max().reset_index(level=0, drop=True)
listings_df['max_lag'].fillna(listings_df['number_of_reviews'], inplace=True)
listings_df.loc[:, 'fixed_lag'] = listings_df.groupby('id')['max_lag'].shift(1)
listings_df.loc[:, "NOR_diff"] = listings_df['max_lag'] - listings_df['fixed_lag']

In [35]:
listings_df['NOR_diff'].describe()

count    136106.000000
mean          2.771597
std           3.665230
min          -4.000000
25%           0.000000
50%           1.000000
75%           4.000000
max          39.000000
Name: NOR_diff, dtype: float64

In [41]:
listings_df[['id', 'number_of_reviews']]

,id,number_of_reviews
0,7893,NaN
1,7893,NaN
2,7893,NaN
3,7893,2.0
4,7893,NaN
...,...,...
418354,32164834,NaN
418355,32164834,NaN
418356,32164834,NaN
418357,32164834,NaN


In [50]:
listings_df.groupby(['id'])['number_of_reviews'].rolling(19, min_periods=1).max().reset_index(level=0, drop=True)

0         NaN
1         NaN
2         NaN
3         2.0
4         2.0
         ... 
418354    NaN
418355    NaN
418356    NaN
418357    NaN
418358    0.0
Name: number_of_reviews, Length: 418359, dtype: float64

In [124]:
listings_df['NOR_diff'].describe()

count    228176.000000
mean          1.705328
std           3.274388
min          -1.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         149.000000
Name: NOR_diff, dtype: float64

In [37]:
listings_df.loc[:, 'corrected_NOR'] = listings_df.groupby(['id'])['number_of_reviews'].rolling(19, min_periods=1).max().reset_index(level=0, drop=True)
listings_df = create_lagsleads('corrected_NOR', 12, listings_df, "NOR") 
listings_df.loc[:, "NOR_diff"] = listings_df['corrected_NOR'] - listings_df['NORlag1']

In [125]:
# Run one more time. I'm not positive if this fixes everthing
listings_df.loc[:, "corrected_NOR2"] = listings_df.groupby(['id'])['corrected_NOR'].rolling(19, min_periods=1).max().reset_index(level=0, drop=True)
listings_df = create_lagsleads('corrected_NOR2', 2, listings_df, "NORcorr") 
listings_df.loc[:, "NOR_diff2"] = listings_df['corrected_NOR2'] - listings_df['NORcorrlag1']

In [126]:
listings_df['NOR_diff'].describe()

count    228176.000000
mean          1.705328
std           3.274388
min          -1.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         149.000000
Name: NOR_diff, dtype: float64

In [127]:
listings_df['NOR_diff2'].describe()

count    253504.000000
mean          1.537246
std           3.161950
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         149.000000
Name: NOR_diff2, dtype: float64

In [105]:
listings_df[listings_df['id'] == 6781814][['number_of_reviews', 'corrected_NOR','NOR_diff']]

,number_of_reviews,corrected_NOR,NOR_diff
97828,1.0,1.0,NaN
97829,1.0,1.0,0.0
97830,1.0,1.0,0.0
97831,1.0,1.0,0.0
97832,1.0,1.0,0.0
97833,1.0,1.0,0.0
97834,1.0,1.0,0.0
97835,1.0,1.0,0.0
97836,2.0,2.0,1.0
97837,NaN,2.0,0.0


In [118]:
demo_df = listings_df[listings_df['id'] == 6781814].copy()
demo_df = demo_df.reset_index(drop=True)

In [123]:
demo_df.groupby(['id'])['corrected_NOR'].rolling(19, min_periods=1).max().reset_index(level=0, drop=True)

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     2.0
9     2.0
10    2.0
11    2.0
12    2.0
13    2.0
14    2.0
15    2.0
16    2.0
17    2.0
18    2.0
19    2.0
20    2.0
21    2.0
22    2.0
23    2.0
24    2.0
25    2.0
26    2.0
27    2.0
28    2.0
29    2.0
30    2.0
31    2.0
32    2.0
33    2.0
34    2.0
35    2.0
36    2.0
Name: corrected_NOR, dtype: float64

In [122]:
demo_df[['number_of_reviews', 'corrected_NOR', 'NOR_diff']]

,number_of_reviews,corrected_NOR,NOR_diff
0,1.0,1.0,NaN
1,1.0,1.0,0.0
2,1.0,1.0,0.0
3,1.0,1.0,0.0
4,1.0,1.0,0.0
5,1.0,1.0,0.0
6,1.0,1.0,0.0
7,1.0,1.0,0.0
8,2.0,2.0,1.0
9,NaN,2.0,0.0


In [114]:
listings_df[listings_df['corrected_NOR'] < listings_df['NORlag12']]

,id,month,List_month,last_scraped,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,...,NORlag3,NORlag4,NORlag5,NORlag6,NORlag7,NORlag8,NORlag9,NORlag10,NORlag11,NORlag12
97855,6781814,27,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
97856,6781814,28,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
97857,6781814,29,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
97858,6781814,30,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
97859,6781814,31,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
97860,6781814,32,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
97861,6781814,33,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
97862,6781814,34,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0
97863,6781814,35,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0
97864,6781814,36,0,NaT,30820.0,Buff Medb,2009-08-10,"Portland, Oregon, United States",NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0


In [100]:
# Example of what the corrected_NOR variable does. Can see that I don't allow the stock of reviews to decline.

listings_df[listings_df['id'] == 10886050][['id', 'last_scraped', 'number_of_reviews', 'corrected_NOR', 'NOR_diff']].iloc[-14:]

,id,last_scraped,number_of_reviews,corrected_NOR,NOR_diff
159456,10886050,2017-11-13,55.0,55.0,4.0
159457,10886050,2017-12-09,51.0,55.0,0.0
159458,10886050,2018-01-16,51.0,55.0,0.0
159459,10886050,2018-02-08,51.0,55.0,0.0
159460,10886050,2018-04-11,59.0,59.0,4.0
159461,10886050,2018-05-14,61.0,61.0,2.0
159462,10886050,2018-07-10,70.0,70.0,9.0
159463,10886050,2018-08-14,78.0,78.0,8.0
159464,10886050,2018-09-14,81.0,81.0,3.0
159465,10886050,2018-10-09,84.0,84.0,3.0


In [38]:
listings_df['NOR_diff'].describe()

count    228176.000000
mean          1.705328
std           3.274388
min          -1.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         149.000000
Name: NOR_diff, dtype: float64

In [ ]:
Confidence_cutoff = 1.960 
df.loc[:, 'corrected_NOR'] = df.groupby(['id'])['number_of_reviews'].rolling(19, min_periods=1).max().reset_index(level=0, drop=True)
df = create_lagsleads('corrected_NOR', 12, df, "NOR") 
df.loc[:, "NOR_diff"] = df['corrected_NOR'] - df['NORlag1']
bounds = df.groupby('id')['NOR_diff'].mean() + Confidence_cutoff*df.groupby('id')['NOR_diff'].std()/np.sqrt(df.groupby('id')['List_month'].sum())
df.loc[:, "NOR_diff_bound"] = bounds[df['id']].values

----------

# Exploring missing data

In [11]:
def checking_missing_data(var):
    
    """
    This function checks what the difference between the number of unique ids and the 
    number of ids paired with some property characteristic. 
    The key use case is to see whether or not a listing trait changes.
    """
    
    ids = listings_df[['id']].drop_duplicates().dropna()
    ids = np.array(ids)

    paired_ids = listings_df[['id', var]].dropna().drop_duplicates()
    paired_ids = np.array(paired_ids)
    
    if len(ids) == len(paired_ids):
        print('No change in variable')
    else:
        print("Variable changes")

In [12]:
def identify_variable_changes(var, cutoff, relate, df):
    
    """
    This function lists ids where the variable of interest changes ("var").
    This can be used for data cleaning purposes.
    """
    
    ops = {'>': operator.gt,
       '<': operator.lt,
       '>=': operator.ge,
       '<=': operator.le,
       '==': operator.eq}
    
    # Take ids and variable of interest and drop any na's
    repetition_arr = np.array(df[['id', var]].dropna().drop_duplicates()) # Need drop_duplicates to identify actual price changes
    counts = np.unique(repetition_arr[:,0], return_counts = True)
    return counts[0][ops[relate](counts[1], cutoff)], counts[1][ops[relate](counts[1], cutoff)]

----------

## Filling in N/A's

This section of the code fills missing data in forwards and backwards for values that would be expected to be invariant over time, such as fixed property features, and host characteristics, such as when the host first started using Airbnb.

In [13]:
# Take the modal zip codes as a property's zip code
modal_zips = listings_df.groupby('id')['zipcode'].agg(lambda x: stats.mode(x)[0][0])
listings_df.loc[:, 'zipcode'] = modal_zips[listings_df['id']].values

### Assumption: Properties do not change features when they are no observed
Forward fill, then back fill. This means older features have priority. This is an assumption! 

In [14]:
# Run a loop that fills property characteristics forwards and backwards

my_timer = time.time() # Time it

for var in ['host_id', 'host_name', 'host_since', 'host_location', 'property_type', 'room_type', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'first_review', 'instant_bookable']:
    listings_df.loc[:, var] = listings_df.groupby(['id'])[var].fillna(method='ffill', axis=0)
    listings_df.loc[:, var] = listings_df.groupby(['id'])[var].fillna(method='bfill', axis=0)
    
time_to_run_filler =  time.time() - my_timer

print("Minutes to run filling loop:")
print(time_to_run_filler/60)

Minutes to run filling loop:
0.9600584348042805


In [15]:
def create_dummies(var):
    """
    Creates a dummy variable for a given variable
    """
    listings_df.loc[:, var] = listings_df[var].astype('category')
    listings_df.loc[:, var + "_dum"] = listings_df[var].cat.codes

In [16]:
# Creates dummy variables for specified categories

categorical_vars = ['host_is_superhost', 'room_type', 'instant_bookable', 'zipcode']

for cat in categorical_vars:
    create_dummies(cat)

-----------

# Neighborhood categorization
This section assigns each property to a specific neighborhood.

### Finding appropriate neighborhoods

In [17]:
# How many total listings there are

print("Total listings:")

print(len(listings_df['neighbourhood'].astype('category')))

print("-------------------------")
print("Active listings with neighborhood reported:")
print(len(listings_df[(listings_df['List_month']==1) & (~listings_df['neighbourhood'].isna())]))

print("-------------------------")
print("Active listings with no neighborhood reported:")
print(len(listings_df[(listings_df['List_month']==1) & (listings_df['neighbourhood'].isna())]))

Total listings:
418359
-------------------------
Active listings with neighborhood reported:
140237
-------------------------
Active listings with no neighborhood reported:
10201


In [18]:
# This code replaces each listing's neighborhood with its modal neighborhood
modal_neighs = listings_df.groupby('id')['neighbourhood'].agg(lambda x: stats.mode(x)[0][0])
listings_df.loc[:, 'neighbourhood'] = modal_neighs[listings_df['id']].values

# Creates a dummy for whether neighborhood was initially reported
listings_df.loc[:, 'missing_neigh'] = (listings_df['neighbourhood'] == 0)*1

### Average the reported longitude and latitude
The longitude and latitude of a given property sometimes changes due to anonymization of Airbnb exact location. We just average these longitudes and latitudes to determine a representative location. 

In [19]:
id_avg_lat, id_avg_lon = listings_df.groupby('id')['latitude'].mean(), listings_df.groupby('id')['longitude'].mean()

listings_df.loc[:,'avg_lat'] = np.array(id_avg_lat[(listings_df['id'].values)])
listings_df.loc[:, 'avg_lon'] = np.array(id_avg_lon[(listings_df['id'].values)])

In [20]:
# Set-up a dataframe of all of the listings missing a neighborhood
missing_neigh = listings_df[(listings_df['List_month'] == 1) & (listings_df['neighbourhood'] == 0) & (~listings_df['latitude'].isna()) & (~listings_df['longitude'].isna())][['id', 'avg_lat', 'avg_lon']]
missing_neigh = missing_neigh.drop_duplicates()
missing_neigh = missing_neigh.sort_index()
missing_neigh = missing_neigh.reset_index(drop = False)

# Dataframe of all the listings, that are not missing a neighborhood
not_missing_neigh = listings_df[(listings_df['List_month'] == 1) & (listings_df['neighbourhood'] != 0) & (~listings_df['latitude'].isna()) & (~listings_df['longitude'].isna())][['id', 'neighbourhood','avg_lat', 'avg_lon']]
not_missing_neigh = not_missing_neigh.drop_duplicates()

# Print first five rows of the missing neighborhood dataframe
missing_neigh.head(5)

,index,id,avg_lat,avg_lon
0,1324,67036,45.531952,-122.644825
1,2109,107177,45.496996,-122.745876
2,4516,246398,45.479189,-122.607044
3,5322,281285,45.532832,-122.705484
4,11027,668224,45.532556,-122.777067


In [21]:
def distance(point1, point2):
    """
    Calculate distance between two points
    """
    return mpu.haversine_distance(point1, point2)

print(distance((30.170165, -97.756954), (30.277500,-97.713975))/km_per_mi)

def closest(data, this_point):
    """
    Applies the distance function to each element in the data, 
    then returns the observation with the lowest distance.
    """
    return min(data, key=lambda x: distance(this_point,x))

7.8474924451354475


In [22]:
coords_with_neigh = np.array(not_missing_neigh[['avg_lat', 'avg_lon']])
coords_no_neigh = np.array(missing_neigh[['avg_lat', 'avg_lon']])

### Neighborhood identification

In [23]:
# Identify neighborhoods for properties with no neighborhood assigned
# NOTE: IF THIS HAS BEEN RUN ONCE, CAN COMMENT OUT AND JUST LOAD IN 'approximated_neighs.csv'

neigh_timer = time.time()

approx_neighs = []

for i in coords_no_neigh:
     approx_neighs.append(not_missing_neigh[(not_missing_neigh['avg_lat'] == closest(tuple(coords_with_neigh), tuple(i))[0]) & (not_missing_neigh['avg_lon'] == closest(tuple(coords_with_neigh), tuple(i))[1])]['neighbourhood'].values[0])

missing_neigh['neighbourhood'] = approx_neighs

# Save the approximate neighborhoods so this code doesn't need to be run again.
missing_neigh.to_csv(city_abbrev + '_approximated_neighs.csv', index=False)

time_to_match = time.time() - neigh_timer

print("Mins to match neighborhoods:")
print(time_to_match/60)

Mins to match neighborhoods:
1.1829299330711365


In [24]:
# If approximated_neighs.csv exists, can load in neighborhoods here.
missing_neigh['neighbourhood'] = pd.read_csv(city_abbrev + '_approximated_neighs.csv')['neighbourhood']

In [25]:
listings_df.loc[:, 'neighbourhood'][missing_neigh['index'].values] = missing_neigh['neighbourhood'].values.copy()

/var/folders/nx/l6t8st49137cl77yqnxy4z2m0000gn/T/ipykernel_27434/369083258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df.loc[:, 'neighbourhood'][missing_neigh['index'].values] = missing_neigh['neighbourhood'].values.copy()


In [26]:
# Replace all of the 0's with NaN's
listings_df.loc[:, 'neighbourhood'] = listings_df['neighbourhood'].replace({0: np.nan})

# Copy the neighborhood over the whole sample
listings_df.loc[:,'neighbourhood'] = listings_df.groupby(['id'])['neighbourhood'].fillna(method='ffill', axis=0)
listings_df.loc[:, 'neighbourhood'] = listings_df.groupby(['id'])['neighbourhood'].fillna(method='bfill', axis=0)

# Create neighborhood dummies
create_dummies('neighbourhood')

-------

# Calendar Formatting

Here we calculate a numeric value for the days since an Airbnb listing's calendar has been updated. This offers a measure for how active the Airbnb property is.

In [27]:
cal_update = listings_df['calendar_updated'].str.split(" ", n=2, expand=True)
cal_update.columns = ['count', 'units', 'numeric']

cal_update = cal_update[['count', 'units']] # Drop the third column
cal_update.loc[:, "count"] = cal_update['count'].replace({"today": 0, "a":1, "yesterday":0, "never":9999}).astype(float)
cal_update.loc[:, "units"] = cal_update['units'].replace({"days": 1, "None":1, "weeks":7, "months":30, "week":7}).astype(float)
cal_update.loc[:, 'days'] = cal_update['count']*cal_update['units']

cal_update.loc[cal_update['count']==0.0, "days"] = 0.0
cal_update.loc[cal_update['count']==9999, "days"] = 9999

# Add the calendar update values to the dataframe
listings_df.loc[:, 'days_since_calup'] = cal_update['days']
del cal_update

In [28]:
# Drop the missing neighborhoods flag
listings_df = listings_df.drop(columns=['missing_neigh'])

____

# Save to csv

In [29]:
os.chdir(csv_save_path)
listings_df.to_csv(city_abbrev + '_1stStageClean.csv.gz', compression='gzip', index=False, date_format='%Y-%m-%d %H:%M:%S')